In [1]:
import os 
os.environ["SPARK_HOME"] = ""

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col, avg

spark = SparkSession.builder\
    .appName("capstone_project")\
    .enableHiveSupport()\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/23 20:16:02 WARN Utils: Your hostname, pcminh-ThinkPad resolves to a loopback address: 127.0.1.1; using 172.31.163.12 instead (on interface eth0)
23/05/23 20:16:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 20:16:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
from schemas import redlight_cam_schema, speed_cam_schema, streets_schema, traffic_hist_schema

tables = [
    'crashes', 
    'redlight_cam', 
    'speed_cam', 
    'streets', 
    'traffic_hist'
]

paths = {
    table: f'./data/raw/traffic/{table}/{table}.csv'
    for table in tables
} 

schemas = {
    # 'crashes'
    'redlight_cam': redlight_cam_schema,
    'speed_cam': speed_cam_schema,
    'streets': streets_schema,
    'traffic_hist': traffic_hist_schema
}

def read_table_to_view(spark: SparkSession):
    for table in tables:
        read_obj = spark.read.format("csv").option("header", "true")

        if table in schemas:
            read_obj = read_obj.schema(schemas[table])
        else:
            read_obj = read_obj.option("inferSchema", "true")
 
        df = read_obj.load(paths[table])
        df.createOrReplaceTempView(table)

In [5]:
read_table_to_view(spark)

23/05/23 20:16:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
spark.sql("SHOW TABLES").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |     crashes|       true|
|         |redlight_cam|       true|
|         |   speed_cam|       true|
|         |     streets|       true|
|         |traffic_hist|       true|
+---------+------------+-----------+



In [23]:
spark.sql("""
select 
    row_number() over (order by street asc, suffix asc, direction asc) as street_key,
    full_street_name as full_street_name,
    direction as direction,
    street as street,
    suffix_direction as suffix_direction,
    min_address as min_address,
    max_address as max_address
from streets
""").show()

+----------+----------------+---------+------+----------------+-----------+-----------+
|street_key|full_street_name|direction|street|suffix_direction|min_address|max_address|
+----------+----------------+---------+------+----------------+-----------+-----------+
|         1|      E 100TH PL|        E| 100TH|                |          1|       1199|
|         2|      W 100TH PL|        W| 100TH|                |        300|       2629|
|         3|      E 100TH ST|        E| 100TH|                |          1|       4001|
|         4|      W 100TH ST|        W| 100TH|                |          1|       2799|
|         5|      E 101ST PL|        E| 101ST|                |          1|        699|
|         6|      W 101ST PL|        W| 101ST|                |        300|       3999|
|         7|      E 101ST ST|        E| 101ST|                |          1|       3699|
|         8|      W 101ST ST|        W| 101ST|                |          1|       3999|
|         9|      E 102ND PL|   

23/05/23 00:59:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/23 00:59:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/23 00:59:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/23 00:59:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Full Street Name, Direction, Street , Suffix, Suffix Direction, Min Address, Max Address
 Schema: full_street_name, direction, street, suffix, suffix_direction, min_address, max_address
Expected: full_street_name but found: Full Street Name
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv


23/05/23 09:57:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 25916029 ms exceeds timeout 120000 ms
23/05/23 09:57:47 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/23 09:57:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage

In [7]:
spark.sql("""
    select distinct
        camera_id,
        "speed" as type,
        NULL as red_light_intersection,
        address,
        split(address, ' ') as addr_toks, 
        CAST(addr_toks[0] AS INT) as addr_no, 
        concat_ws(' ', filter(addr_toks , (x,i) -> i > 0)) as street_name,
        latitude,
        longitude
    from speed_cam
""").show()

23/05/22 23:41:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ADDRESS, CAMERA ID, LATITUDE, LONGITUDE
 Schema: address, camera_id, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv


+---------+-----+----------------------+--------------------+--------------------+-------+----------------+------------+-------------+
|camera_id| type|red_light_intersection|             address|           addr_toks|addr_no|     street_name|    latitude|    longitude|
+---------+-----+----------------------+--------------------+--------------------+-------+----------------+------------+-------------+
|   CHI157|speed|                  null|   1754 N PULASKI RD|[1754, N, PULASKI...|   1754|    N PULASKI RD|41.913374831|  -87.7265886|
|   CHI014|speed|                  null|       6909 S KEDZIE|   [6909, S, KEDZIE]|   6909|        S KEDZIE|41.767731869|-87.702737907|
|   CHI111|speed|                  null|    2326 W CERMAK RD|[2326, W, CERMAK,...|   2326|     W CERMAK RD|41.852153505|-87.684338389|
|   CHI070|speed|                  null|         2513 W 55TH|     [2513, W, 55TH]|   2513|          W 55TH|41.793702249|-87.687244272|
|   CHI083|speed|                  null|6226 W IRVING P

In [18]:
spark.sql("""
SELECT DISTINCT camera_id, "speed" as type, NULL as red_light_intersection, address, latitude, longitude FROM speed_cam
""").show()

23/05/22 22:56:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ADDRESS, CAMERA ID, LATITUDE, LONGITUDE
 Schema: address, camera_id, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv


+---------+-----+----------------------+--------------------+------------+-------------+
|camera_id| type|red_light_intersection|             address|    latitude|    longitude|
+---------+-----+----------------------+--------------------+------------+-------------+
|   CHI128|speed|                  null|  1226 N WESTERN AVE|41.903817103|-87.687194998|
|   CHI010|speed|                  null|     1111 N HUMBOLDT|        null|         null|
|   CHI154|speed|                  null|     2416 W 103RD ST|41.706487201|-87.682391059|
|   CHI180|speed|                  null|7157 S SOUTH CHIC...|41.764775432|-87.603827682|
|   CHI141|speed|                  null|      3542 E 95TH ST| 41.72304462|-87.537040758|
|   CHI020|speed|                  null|    2912 W ROOSEVELT|41.866585283|-87.698962375|
|   CHI058|speed|                  null|      5816 W JACKSON|41.877189405|-87.770369502|
|   CHI068|speed|                  null|      5529 S WESTERN|        null|         null|
|   CHI162|speed|    